# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [8]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2017/09	bomrang: Fetch Australian Government Bureau of Meteorology Data in R	The Journal of Open Source Software	The Australian Bureau of Meteorology (BoM) publicly provides data via an anonymous FTP server, in XML and json files (Australian Bureau of Meteorology 2017). The files are well structured but knowledge of how to use R and parse them is required to extract the data into a data frame for use in R (R Core Team 2017) or requires external programs and scripting to import the data for use. bomrang provides functionality for automated retrieval and parsing of selected weather data files from BoM.	Sparks AH, Padgham M, Parsonage H and Pembleton K (2017). “bomrang: Fetch Australian Government Bureau of Meteorology Weather Data.” The Journal of Open Source Software, 2(17). doi: 10.21105/joss.00411.	sparks_et_al_2017a	http://joss.theoj.org/papers/350bf005bded599e4b0f3ac2acf138e8
2017/07/19	Mungbean and sorghum disease update	GRDC

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

ImportError: Missing required dependencies ['numpy']

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


NameError: name 'pd' is not defined

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

NameError: name 'publications' is not defined

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

01-cheatham_et_al_2009.md 18-sparks_et_al_2011.md   31-esker_et_al_2007b.md
01-savary_et_al_2011.md   19-duku_et_al_2016.md     31-garrett_et_al_2007.md
05-dosa_et_al_2015.md     19-kelly_et_al_2017.md    31-mckinley_et_al_2012.md
10-barnwal_et_al_2013.md  28-laborte_et_al_2015.md  31-sparks_2017.md
10-garrett_et_al_2011.md  28-sparks_et_al_2017.md   31-sparks_et_al_2008a.md
12-gaudin_et_al_2012.md   30-sparks_et_al_2014.md   31-sparks_et_al_2008b.md
17-savary_et_al_2017.md   31-esker_et_al_2007a.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
